In [1]:
import garmin_cookies
client = garmin_cookies.main()

INFO:garmin_cookies:Successfully logged in to Garmin Connect


In [42]:
def extract_laps_to_df(activities_list):
    """
    Extract lapDTOs from multiple activities into a single DataFrame
    """
    all_laps = []
    
    for activity in activities_list:
        if 'lapDTOs' in activity and activity['lapDTOs']:
            for i, lap in enumerate(activity['lapDTOs'], 1):
                # Add activity metadata to each lap
                lap_with_metadata = lap.copy()
                lap_with_metadata['activityId'] = activity['activityId']
                lap_with_metadata['lap_number'] = i
                all_laps.append(lap_with_metadata)
    
    # Create DataFrame
    laps_df = pd.DataFrame(all_laps)
    
    # Convert datetime columns
    datetime_columns = ['startTimeGMT', 'startTimeLocal', 'endTimeGMT']
    for col in datetime_columns:
        if col in laps_df.columns:
            laps_df[col] = pd.to_datetime(laps_df[col])
    
    return laps_df

# Usage:
laps_df = extract_laps_to_df([activity_data])
laps_df


,startTimeGMT,startLatitude,startLongitude,distance,duration,movingDuration,elapsedDuration,elevationGain,elevationLoss,maxElevation,...,maxVerticalSpeed,directWorkoutComplianceScore,avgGradeAdjustedSpeed,lapIndex,lengthDTOs,connectIQMeasurement,intensityType,messageIndex,activityId,lap_number
0,2025-10-01 05:13:57,48.881933,2.274410,1000.00,336.689,335.000,336.689,5.0,0.0,45.4,...,0.200001,100,2.995,1,[],[],INTERVAL,0,20553665052,1
1,2025-10-01 05:19:34,48.876429,2.270368,1000.00,348.435,348.435,348.435,7.0,1.0,51.6,...,0.200001,100,2.915,2,[],[],INTERVAL,1,20553665052,2
2,2025-10-01 05:25:23,48.872384,2.275722,1000.00,325.491,325.000,325.491,0.0,3.0,50.2,...,0.200001,100,3.047,3,[],[],INTERVAL,2,20553665052,3
3,2025-10-01 05:30:48,48.868471,2.264730,1000.00,331.904,331.904,331.904,3.0,2.0,47.8,...,0.200001,100,3.020,4,[],[],INTERVAL,3,20553665052,4
4,2025-10-01 05:36:20,48.861432,2.257869,1000.00,331.977,331.977,331.977,4.0,0.0,52.8,...,0.200001,100,3.046,5,[],[],INTERVAL,4,20553665052,5
5,2025-10-01 05:41:52,48.853359,2.253677,1000.00,329.173,329.000,329.173,8.0,11.0,54.6,...,0.200001,100,3.032,6,[],[],INTERVAL,5,20553665052,6
6,2025-10-01 05:47:21,48.849975,2.258465,1000.00,331.810,331.810,331.810,2.0,2.0,51.8,...,0.200001,100,3.023,7,[],[],INTERVAL,6,20553665052,7
7,2025-10-01 05:52:53,48.858049,2.261926,1000.00,320.068,320.000,320.068,0.0,3.0,49.2,...,0.200001,100,3.098,8,[],[],INTERVAL,7,20553665052,8
8,2025-10-01 05:58:13,48.864764,2.262827,1000.00,316.346,316.000,316.346,0.0,4.0,45.4,...,0.200001,100,3.127,9,[],[],INTERVAL,8,20553665052,9
9,2025-10-01 06:03:29,48.871161,2.271635,1000.00,328.067,328.000,328.067,2.0,8.0,44.4,...,0.200001,100,3.042,10,[],[],INTERVAL,9,20553665052,10


In [31]:


# Connect to SQLite (creates a file if it doesn't exist)
conn = sqlite3.connect("activities.db")

# Save DataFrame to SQLite
df = pd.read_csv("data/processed/activities_2025_09_04.csv", delimiter=';')  # Use the correct delimiter
df.drop(columns=['Unnamed: 0']).to_sql("activities", conn, if_exists="replace", index=False)

# Close connection
conn.close()

In [23]:
import sqlite3
conn = sqlite3.connect("activities.db")
import pandas as pd 
pd.set_option('display.max_columns', None)

# Query with a condition
query = """
   WITH RECURSIVE date_series AS (
            SELECT date("now", "-84 days", "weekday 1") AS Week
            UNION ALL
            SELECT date(Week, '+7 days')
            FROM date_series
            WHERE date(Week, '+7 days') < date("now", "weekday 1")
        )

         
        
         SELECT
            ds.*,
            a.Day,
            a.startTimeLocal,
            a.locationName,
            a.activityId,
            a.distance,
            a.duration, --time(ds.duration, 'unixepoch') as duration,
            a.calories,
            a.averageHR,
            a.elevationGain,
            a.averageSpeed*3.6 as averageSpeed,
            a.averageRunCadence,
            a.elevationLoss,
            a.trainingEffect,
            a.trainingEffectLabel,
            a.maxHR,
            a.minHR
        FROM date_series ds 

        LEFT JOIN activities a ON strftime('%Y-%m-%d', ds.Week) = strftime('%Y-%m-%d', date(a.Week, 'weekday 1'))
         AND a.activityTypeGrouped = 'running'
        ORDER BY ds.Week;

"""
df_filtered = pd.read_sql(query, conn)
df_filtered

,Week,Day,startTimeLocal,locationName,activityId,distance,duration,calories,averageHR,elevationGain,averageSpeed,averageRunCadence,elevationLoss,trainingEffect,trainingEffectLabel,maxHR,minHR
0,2025-08-04,2025-08-04,2025-08-04 17:51:05,San Francisco,1.995383e+10,16.52775,4489.462,1192.0,158.0,136.0,13.251600,165.156250,138.0,4.2,LACTATE_THRESHOLD,187.0,53.0
1,2025-08-04,2025-08-06,2025-08-06 17:58:59,San Francisco,1.997709e+10,16.00276,4666.008,1157.0,153.0,88.0,12.348000,156.609375,86.0,3.9,ANAEROBIC_CAPACITY,198.0,68.0
2,2025-08-04,2025-08-09,2025-08-09 12:36:23,San Francisco,2.000642e+10,13.99719,3840.794,1018.0,160.0,120.0,13.118400,163.937500,119.0,3.9,LACTATE_THRESHOLD,184.0,76.0
3,2025-08-11,2025-08-11,2025-08-11 17:51:00,San Francisco,2.006738e+10,15.01826,4543.682,1048.0,132.0,115.0,11.898000,160.687500,110.0,3.1,AEROBIC_BASE,160.0,59.0
4,2025-08-11,2025-08-13,2025-08-13 18:01:45,San Francisco,2.006738e+10,18.00123,4886.878,1261.0,152.0,97.0,13.262400,162.796875,89.0,4.2,VO2MAX,194.0,69.0
5,2025-08-11,2025-08-15,2025-08-15 07:13:41,San Francisco,2.007248e+10,12.02079,3752.955,842.0,133.0,108.0,11.530800,163.234375,142.0,2.8,AEROBIC_BASE,161.0,90.0
6,2025-08-11,2025-08-16,2025-08-16 12:00:20,San Francisco,2.008092e+10,13.01796,3626.385,936.0,153.0,102.0,12.924000,164.796875,103.0,3.4,AEROBIC_BASE,169.0,67.0
7,2025-08-18,2025-08-18,2025-08-18 18:33:49,San Francisco,2.010394e+10,6.00437,1498.764,408.0,154.0,15.0,14.421600,167.281250,10.0,3.0,TEMPO,167.0,114.0
8,2025-08-18,2025-08-20,2025-08-20 18:01:22,San Francisco,2.012648e+10,17.50440,4680.426,1194.0,150.0,137.0,13.464000,161.750000,80.0,3.9,VO2MAX,193.0,73.0
9,2025-08-18,2025-08-22,2025-08-22 14:47:07,Garfield County,2.014623e+10,16.40000,7325.000,1263.0,133.0,620.0,8.060068,142.796875,620.0,3.0,AEROBIC_BASE,173.0,68.0


In [ ]:
 SELECT
            ds.*,
            a.Day,
            a.startTimeLocal,
            a.locationName,
            a.activityId,
            a.distance,
            a.duration, --time(ds.duration, 'unixepoch') as duration,
            a.calories,
            a.averageHR,
            a.elevationGain,
            a.averageSpeed*3.6 as averageSpeed,
            a.averageRunCadence,
            a.elevationLoss,
            a.trainingEffect,
            a.trainingEffectLabel,
            a.maxHR,
            a.minHR
        FROM date_series ds 

        LEFT JOIN activities a ON strftime('%Y-%m-%d', ds.Week) = strftime('%Y-%
m-%d', date(a.Week, 'weekday 1'))
         AND a.activityTypeGrouped = 'running'
        ORDER BY ds.Week;

In [1]:
import sqlite3
conn = sqlite3.connect("activities.db")

# Query with a condition
query = """

       select 
       * from activities

    """
df_filtered = pd.read_sql(query, conn)
df_filtered

NameError: name 'pd' is not defined